In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25517415
paper_name = 'winter_curtin_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table_S1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 218 x 3


In [9]:
original_data.head()

,ORF,Name,PHENOTYPE,orfs
0,YNL141W,AAH1,LOW,YNL141W
1,YOR239W,ABP140,MODERATELY LOW,YOR239W
2,YLR144C,ACF2,MODERATELY HIGH,YLR144C
3,YAR015W,ADE1,LOW,YAR015W
4,YOR128C,ADE2,LOW,YOR128C


In [10]:
original_data['orfs'] = original_data['ORF'].astype(str)

In [11]:
# Eliminate all white spaces & capitalize
original_data['orfs'] = clean_orf(original_data['orfs'])

In [12]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['orfs'], to='orf')

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Name, PHENOTYPE, orfs]
Index: []


In [14]:
data_dict = {'LOW': -2, 'MODERATELY LOW': -1, 'MODERATELY HIGH': 1, 'HIGH': 2}

In [15]:
original_data['data'] = original_data['PHENOTYPE'].apply(lambda x: data_dict[x])

In [16]:
original_data.set_index('orfs', inplace=True)

In [17]:
original_data.index.name='orf'

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(217, 1)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [16551]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,16551
data_type,value
orf,
YAL021C,-2
YAL024C,-2
YAL028W,-1
YAR015W,-2
YAR018C,-1


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16551
,data_type,value
gene_id,orf,
19,YAL021C,-2
22,YAL024C,-2
26,YAL028W,-1
67,YAR015W,-2
68,YAR018C,-1


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16551          
data_type       value    valuez
gene_id orf                    
19      YAL021C    -2 -5.343532
22      YAL024C    -2 -5.343532
26      YAL028W    -1 -2.644503
67      YAR015W    -2 -5.343532
68      YAR018C    -1 -2.644503

# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 25517415...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.79it/s]

Updating the data_modified_on field...
